# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [41]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///../../../db/dm_eyf.db

In [2]:
dataset_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/datasets/'
dataset_file = 'competencia_01.csv'

In [3]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("../../../datasets/competencia_01.csv")

,Success


In [3]:
df=pd.read_csv(dataset_path + dataset_file)

In [4]:
campos=df.columns.to_list()
print(campos[-1])
campos.pop(-1) #elimino clase ternaria
print(campos[-1])

clase_ternaria
Visa_mpagominimo


Ahora creo una tabla con las nuevas columnas, tengo que hacer la macro por que estoy dividiendo por 0.. 

In [5]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0)

,Success


In [6]:
%%sql
CREATE OR REPLACE MACRO div_sin_cero(a, b) as
    case
        when ifnull(b, 0) = 0 THEN NULL
        else ifnull(a, 0) / ifnull(b, 1)
    end

,Success


In [7]:
# Separar las columnas por tipo
master = [col for col in campos if 'Master' in col]
visa = [col for col in campos if 'Visa' in col]

sumas = []

for m_col, v_col in zip(master, visa):
    # Crear la suma usando la función 'suma_sin_null'
    suma = f"\n    suma_sin_null({m_col}, {v_col}) as {m_col.replace('Master_', '')}_total"
    sumas.append(suma)

consulta_sql = f"""
select 
    numero_de_cliente,
    {', '.join(sumas)}
from competencia_01;
"""

print(consulta_sql)


select 
    numero_de_cliente,
    
    suma_sin_null(Master_delinquency, Visa_delinquency) as delinquency_total, 
    suma_sin_null(Master_status, Visa_status) as status_total, 
    suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as mfinanciacion_limite_total, 
    suma_sin_null(Master_Fvencimiento, Visa_Fvencimiento) as Fvencimiento_total, 
    suma_sin_null(Master_Finiciomora, Visa_Finiciomora) as Finiciomora_total, 
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as msaldototal_total, 
    suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as msaldopesos_total, 
    suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as msaldodolares_total, 
    suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as mconsumospesos_total, 
    suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as mconsumosdolares_total, 
    suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as mlimitecompra_total, 
    suma_sin_null(Master_madelantopesos, Vi

In [8]:
%%sql
create or replace table competencia_01_suma_division as
select
    numero_de_cliente,
    foto_mes,
    suma_sin_null(Master_delinquency, Visa_delinquency) as delinquency_total,
    suma_sin_null(Master_status, Visa_status) as status_total,
    suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as mfinanciacion_limite_total,
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as msaldototal_total,
    suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as msaldopesos_total,
    suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as msaldodolares_total,
    suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as mconsumospesos_total,
    suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as mconsumosdolares_total,
    suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as mlimitecompra_total,
    suma_sin_null(Master_madelantopesos, Visa_madelantopesos) as madelantopesos_total,
    suma_sin_null(Master_madelantodolares, Visa_madelantodolares) as madelantodolares_total,
    suma_sin_null(Master_mpagado, Visa_mpagado) as mpagado_total,
    suma_sin_null(Master_mpagospesos, Visa_mpagospesos) as mpagospesos_total,
    suma_sin_null(Master_mpagosdolares, Visa_mpagosdolares) as mpagosdolares_total,
    suma_sin_null(Master_mconsumototal, Visa_mconsumototal) as mconsumototal_total,
    suma_sin_null(Master_cconsumos, Visa_cconsumos) as cconsumos_total,
    suma_sin_null(Master_cadelantosefectivo, Visa_cadelantosefectivo) as cadelantosefectivo_total,
    suma_sin_null(Master_mpagominimo, Visa_mpagominimo) as mpagominimo_total,

    suma_sin_null(ctarjeta_visa, ctarjeta_master) as cant_tarjetas,
    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as cant_transac_tarjetas,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as mtarjetas_consumo,
    suma_sin_null(cprestamos_personales,cprestamos_prendarios) as c_prestamos,
    suma_sin_null(mpayroll, mpayroll2) as m_payroll_total, -- monto acreditacion
    suma_sin_null(cpayroll_trx, cpayroll2_trx) as m_cpayroll_total, -- cantidad de acreditaciones
    suma_sin_null(minversion1_dolares, minversion1_pesos) as minversion1_total,
    suma_sin_null(mplazo_fijo_dolares, mplazo_fijo_pesos) as mplazofijo_total,
    suma_sin_null(suma_sin_null(cseguro_vida, cseguro_auto), suma_sin_null(cseguro_vivienda, cseguro_accidentes_personales)) as cseguros_total,

    div_sin_cero(mconsumototal_total, mlimitecompra_total) as ratio_uso_credito,
    div_sin_cero(mpagado_total, mconsumototal_total) as ratio_pago_vs_consumo,
    div_sin_cero(mpagominimo_total, mpagado_total) as ratio_pago_min_vs_total,
    div_sin_cero(mconsumospesos_total, mconsumosdolares_total) as ratio_consumo_pesos_dolares,
    div_sin_cero(madelantopesos_total + madelantodolares_total, mlimitecompra_total) as ratio_adelantos_vs_limite,
    div_sin_cero(cseguros_total, mconsumototal_total) as ratio_seguros_vs_consumo,

    greatest(Master_Finiciomora,Visa_Finiciomora) as fechamora_mayor,
    least(Master_Finiciomora,Visa_Finiciomora) as fechamora_menor,
    greatest(Master_fultimo_cierre,Visa_fultimo_cierre) as fechacierre_mayor,
    least(Master_fultimo_cierre,Visa_fultimo_cierre) as fechacierre_menor,
    greatest(Master_Fvencimiento, Visa_Fvencimiento) as fvencimiento_mayor,
    least(Master_Fvencimiento, Visa_Fvencimiento) as fvencimiento_menor,
    greatest(Master_fechaalta, Visa_fechaalta) as fechaalta_mayor,
    least(Master_fechaalta, Visa_fechaalta) as fechalta_menor,

    ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6
from competencia_01


,Success


In [9]:
%%sql
select 
    *
from competencia_01_suma_division


,numero_de_cliente,foto_mes,delinquency_total,status_total,mfinanciacion_limite_total,msaldototal_total,msaldopesos_total,msaldodolares_total,mconsumospesos_total,mconsumosdolares_total,...,ratio_seguros_vs_consumo,fechamora_mayor,fechamora_menor,fechacierre_mayor,fechacierre_menor,fvencimiento_mayor,fvencimiento_menor,fechaalta_mayor,fechalta_menor,cliente_antiguedad_6
0,576634596,202106,0,0,1298112.28,158235.84,175216.98,0.00,78367.77,1.80,...,0.000026,NaN,NaN,7.0,7.0,-183.0,-913.0,5313.0,5313.0,8
1,681808990,202106,0,0,88678.80,9119.07,9900.28,796.38,3173.90,8.19,...,0.000630,NaN,NaN,0.0,0.0,-1310.0,-2040.0,4174.0,4174.0,8
2,684878853,202106,0,0,6429567.70,189326.23,222079.66,0.00,163971.42,6.28,...,0.000000,NaN,NaN,0.0,0.0,-548.0,-1279.0,4945.0,4945.0,8
3,686497329,202106,0,0,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
4,687007886,202106,0,0,253368.00,7762.61,8604.57,500.96,493.41,3.69,...,0.002027,NaN,NaN,0.0,0.0,-1338.0,-1522.0,1428.0,496.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,783904040,202102,0,0,2621276.70,155583.86,182499.86,0.00,174774.65,0.00,...,0.000000,NaN,NaN,-3.0,-3.0,-639.0,-1401.0,4819.0,1564.0,7
981942,784422111,202102,0,0,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
981943,785117545,202102,0,0,63342.00,4153.82,4153.82,0.00,0.00,0.00,...,NaN,NaN,NaN,4.0,4.0,-364.0,-364.0,2562.0,2562.0,7
981944,785196597,202102,0,0,608083.20,16471.23,19320.75,0.00,6024.85,0.00,...,0.000000,NaN,NaN,-3.0,-3.0,-609.0,-1035.0,4512.0,4100.0,7


## lag, valor del mes anterior 
## delta diferencia entre meses

In [10]:
campos = ["ctrx_quarter","cpayroll_trx","mpayroll","mpayroll2","cpayroll2_trx","mpasivos_margen",'mprestamos_personales','mcuentas_saldo','mcaja_ahorro','mcomisiones_mantenimiento',
'cproductos','cdescubierto_preacordado',"mtarjeta_visa_consumo","cprestamos_personales", "mcuenta_corriente"]

In [11]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n    , lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}"
  nuevos_features += f"\n    , {campo} - lag_1_{campo} as delta_1_{campo}"
  nuevos_features += f"\n    , lag({campo}, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_{campo}"
  nuevos_features += f"\n    , {campo} - lag_2_{campo} as delta_2_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_lag_delta AS
select
    {nuevos_features.strip()}
FROM competencia_01
"""


In [45]:
print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_lag_delta AS
select
    , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
    , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
    , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
    , ctrx_quarter - lag_2_ctrx_quarter as delta_2_ctrx_quarter
    , lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
    , cpayroll_trx - lag_1_cpayroll_trx as delta_1_cpayroll_trx
    , lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx
    , cpayroll_trx - lag_2_cpayroll_trx as delta_2_cpayroll_trx
    , lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
    , mpayroll - lag_1_mpayroll as delta_1_mpayroll
    , lag(mpayroll, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll
    , mpayroll - l

In [12]:
%%sql
CREATE OR REPLACE TABLE competencia_01_lag_delta AS
select
    numero_de_cliente
    , foto_mes
    , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
    , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
    , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
    , ctrx_quarter - lag_2_ctrx_quarter as delta_2_ctrx_quarter
    , lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
    , cpayroll_trx - lag_1_cpayroll_trx as delta_1_cpayroll_trx
    , lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx
    , cpayroll_trx - lag_2_cpayroll_trx as delta_2_cpayroll_trx
    , lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
    , mpayroll - lag_1_mpayroll as delta_1_mpayroll
    , lag(mpayroll, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll
    , mpayroll - lag_2_mpayroll as delta_2_mpayroll
    , lag(mpayroll2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll2
    , mpayroll2 - lag_1_mpayroll2 as delta_1_mpayroll2
    , lag(mpayroll2, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll2
    , mpayroll2 - lag_2_mpayroll2 as delta_2_mpayroll2
    , lag(cpayroll2_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll2_trx
    , cpayroll2_trx - lag_1_cpayroll2_trx as delta_1_cpayroll2_trx
    , lag(cpayroll2_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll2_trx
    , cpayroll2_trx - lag_2_cpayroll2_trx as delta_2_cpayroll2_trx
    , lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen
    , mpasivos_margen - lag_1_mpasivos_margen as delta_1_mpasivos_margen
    , lag(mpasivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpasivos_margen
    , mpasivos_margen - lag_2_mpasivos_margen as delta_2_mpasivos_margen
    , lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales
    , mprestamos_personales - lag_1_mprestamos_personales as delta_1_mprestamos_personales
    , lag(mprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_personales
    , mprestamos_personales - lag_2_mprestamos_personales as delta_2_mprestamos_personales
    , lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo
    , mcuentas_saldo - lag_1_mcuentas_saldo as delta_1_mcuentas_saldo
    , lag(mcuentas_saldo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuentas_saldo
    , mcuentas_saldo - lag_2_mcuentas_saldo as delta_2_mcuentas_saldo
    , lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro
    , mcaja_ahorro - lag_1_mcaja_ahorro as delta_1_mcaja_ahorro
    , lag(mcaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro
    , mcaja_ahorro - lag_2_mcaja_ahorro as delta_2_mcaja_ahorro
    , lag(mcomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - lag_1_mcomisiones_mantenimiento as delta_1_mcomisiones_mantenimiento
    , lag(mcomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - lag_2_mcomisiones_mantenimiento as delta_2_mcomisiones_mantenimiento
    , lag(cproductos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cproductos
    , cproductos - lag_1_cproductos as delta_1_cproductos
    , lag(cproductos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cproductos
    , cproductos - lag_2_cproductos as delta_2_cproductos
    , lag(cdescubierto_preacordado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cdescubierto_preacordado
    , cdescubierto_preacordado - lag_1_cdescubierto_preacordado as delta_1_cdescubierto_preacordado
    , lag(cdescubierto_preacordado, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cdescubierto_preacordado
    , cdescubierto_preacordado - lag_2_cdescubierto_preacordado as delta_2_cdescubierto_preacordado
    , lag(mtarjeta_visa_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo
    , mtarjeta_visa_consumo - lag_1_mtarjeta_visa_consumo as delta_1_mtarjeta_visa_consumo
    , lag(mtarjeta_visa_consumo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_visa_consumo
    , mtarjeta_visa_consumo - lag_2_mtarjeta_visa_consumo as delta_2_mtarjeta_visa_consumo
    , lag(cprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_personales
    , cprestamos_personales - lag_1_cprestamos_personales as delta_1_cprestamos_personales
    , lag(cprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cprestamos_personales
    , cprestamos_personales - lag_2_cprestamos_personales as delta_2_cprestamos_personales
    , lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente
    , mcuenta_corriente - lag_1_mcuenta_corriente as delta_1_mcuenta_corriente
    , lag(mcuenta_corriente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_corriente
    , mcuenta_corriente - lag_2_mcuenta_corriente as delta_2_mcuenta_corriente
FROM competencia_01

,Success


In [13]:
%%sql
select 
    *
from competencia_01_lag_delta
limit 10

,numero_de_cliente,foto_mes,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter,delta_2_ctrx_quarter,lag_1_cpayroll_trx,delta_1_cpayroll_trx,lag_2_cpayroll_trx,delta_2_cpayroll_trx,...,lag_2_mtarjeta_visa_consumo,delta_2_mtarjeta_visa_consumo,lag_1_cprestamos_personales,delta_1_cprestamos_personales,lag_2_cprestamos_personales,delta_2_cprestamos_personales,lag_1_mcuenta_corriente,delta_1_mcuenta_corriente,lag_2_mcuenta_corriente,delta_2_mcuenta_corriente
0,249221109,202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249221109,202102,166.0,-5.0,NaN,NaN,2.0,-1.0,NaN,NaN,...,NaN,NaN,0.0,0.0,NaN,NaN,-388.19,388.19,NaN,NaN
2,249221109,202103,161.0,13.0,166.0,8.0,1.0,1.0,2.0,0.0,...,119804.35,-108886.27,0.0,0.0,0.0,0.0,0.00,-485.88,-388.19,-97.69
3,249221109,202104,174.0,14.0,161.0,27.0,2.0,-1.0,1.0,0.0,...,30112.81,-10863.50,0.0,0.0,0.0,0.0,-485.88,485.88,0.00,0.00
4,249221109,202105,188.0,11.0,174.0,25.0,1.0,3.0,2.0,2.0,...,10918.08,10589.53,0.0,0.0,0.0,0.0,0.00,-16.84,-485.88,469.04
5,249221109,202106,199.0,0.0,188.0,11.0,4.0,-3.0,1.0,0.0,...,19249.31,1458.98,0.0,0.0,0.0,0.0,-16.84,16.84,0.00,0.00
6,249278864,202103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,249278864,202104,110.0,0.0,NaN,NaN,1.0,0.0,NaN,NaN,...,NaN,NaN,0.0,0.0,NaN,NaN,0.00,0.00,NaN,NaN
8,249278864,202105,110.0,-16.0,110.0,-16.0,1.0,0.0,1.0,0.0,...,3147.81,21280.77,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
9,249278864,202106,94.0,-16.0,110.0,-32.0,1.0,0.0,1.0,0.0,...,17411.08,10404.56,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00


## Ahora voy con el promedio de cada campo

In [14]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n    ,avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
FROM competencia_01
"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    , foto_mes 
    ,avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctrx_quarter
    ,avg(cpayroll_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll_trx
    ,avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll
    ,avg(mpayroll2) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll2
    ,avg(cpayroll2_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll2_trx
    ,avg(mpasivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpasivos_margen
    ,avg(mprestamos_personales) over (partition by numero_de_c

In [15]:
%%sql
CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    , foto_mes 
    ,avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctrx_quarter
    ,avg(cpayroll_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll_trx
    ,avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll
    ,avg(mpayroll2) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll2
    ,avg(cpayroll2_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll2_trx
    ,avg(mpasivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpasivos_margen
    ,avg(mprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mprestamos_personales
    ,avg(mcuentas_saldo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuentas_saldo
    ,avg(mcaja_ahorro) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcaja_ahorro
    ,avg(mcomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcomisiones_mantenimiento
    ,avg(cproductos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cproductos
    ,avg(cdescubierto_preacordado) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cdescubierto_preacordado
    ,avg(mtarjeta_visa_consumo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mtarjeta_visa_consumo
    ,avg(cprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cprestamos_personales
    ,avg(mcuenta_corriente) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuenta_corriente
FROM competencia_01

,Success


In [16]:
%%sql
select 
    *
from competencia_01_avg

,numero_de_cliente,foto_mes,avg_3_ctrx_quarter,avg_3_cpayroll_trx,avg_3_mpayroll,avg_3_mpayroll2,avg_3_cpayroll2_trx,avg_3_mpasivos_margen,avg_3_mprestamos_personales,avg_3_mcuentas_saldo,avg_3_mcaja_ahorro,avg_3_mcomisiones_mantenimiento,avg_3_cproductos,avg_3_cdescubierto_preacordado,avg_3_mtarjeta_visa_consumo,avg_3_cprestamos_personales,avg_3_mcuenta_corriente
0,249246268,202101,29.000000,0.0,0.000,0.00,0.0,25.580000,0.0,-5735.520,0.00,1364.940,6.0,1.0,228.780000,0.0,-3750.490000
1,249246268,202102,28.000000,0.5,5.865,0.00,0.0,14.105000,0.0,-4393.700,105.72,1364.940,6.0,1.0,228.780000,0.0,-2642.220000
2,249301400,202101,53.000000,0.0,0.000,0.00,0.0,449.340000,0.0,1073.210,7590.00,0.000,7.0,1.0,30106.100000,0.0,-4765.850000
3,249301400,202102,51.500000,0.0,0.000,0.00,0.0,318.090000,0.0,4975.845,3795.00,964.085,7.0,1.0,22545.615000,0.0,-7022.310000
4,249301400,202103,47.666667,0.0,0.000,0.00,0.0,284.366667,0.0,2379.820,2530.00,1478.040,7.0,1.0,17769.726667,0.0,-6813.996667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1595193701,202105,4.000000,1.0,75899.520,75899.52,1.0,0.000000,0.0,0.000,0.00,0.000,7.0,1.0,0.000000,0.0,0.000000
981942,1595193701,202106,22.000000,1.0,76199.170,37949.76,0.5,279.380000,0.0,980.950,12202.94,0.000,6.5,1.0,0.000000,0.0,0.000000
981943,1598206246,202106,5.000000,3.0,95219.600,0.00,0.0,7.340000,0.0,5731.440,573.13,0.000,7.0,1.0,0.000000,0.0,-17.700000
981944,1598444941,202106,2.000000,1.0,23460.000,0.00,0.0,0.000000,0.0,0.000,0.00,0.000,6.0,1.0,0.000000,0.0,0.000000


## reg_slope

In [17]:
nuevos_features = ""
for campo in campos:
  nuevos_features +=  f"\n    ,regr_slope({campo}, cliente_antiguedad) over ventana_3 as {campo}_slope_3"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
    ,regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_3 as cpayroll_trx_slope_3
    ,regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as mpayroll_slope_3
    ,regr_slope(mpayroll2, cliente_antiguedad) over ventana_3 as mpayroll2_slope_3
    ,regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_3 as cpayroll2_trx_slope_3
    ,regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3
    ,regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as mprestamos_personales_slope_3
    ,regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as mcuentas_saldo_slope_3
    ,regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as mcaja_ahorro_slope_3
    ,regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as mco

In [18]:
%%sql
CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
    ,regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_3 as cpayroll_trx_slope_3
    ,regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as mpayroll_slope_3
    ,regr_slope(mpayroll2, cliente_antiguedad) over ventana_3 as mpayroll2_slope_3
    ,regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_3 as cpayroll2_trx_slope_3
    ,regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3
    ,regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as mprestamos_personales_slope_3
    ,regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as mcuentas_saldo_slope_3
    ,regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as mcaja_ahorro_slope_3
    ,regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as mcomisiones_mantenimiento_slope_3
    ,regr_slope(cproductos, cliente_antiguedad) over ventana_3 as cproductos_slope_3
    ,regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_3 as cdescubierto_preacordado_slope_3
    ,regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_3 as mtarjeta_visa_consumo_slope_3
    ,regr_slope(cprestamos_personales, cliente_antiguedad) over ventana_3 as cprestamos_personales_slope_3
    ,regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_3 as mcuenta_corriente_slope_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)

,Success


In [19]:
%%sql
select 
    *
from competencia_01_regr_slope

,numero_de_cliente,foto_mes,ctrx_quarter_slope_3,cpayroll_trx_slope_3,mpayroll_slope_3,mpayroll2_slope_3,cpayroll2_trx_slope_3,mpasivos_margen_slope_3,mprestamos_personales_slope_3,mcuentas_saldo_slope_3,mcaja_ahorro_slope_3,mcomisiones_mantenimiento_slope_3,cproductos_slope_3,cdescubierto_preacordado_slope_3,mtarjeta_visa_consumo_slope_3,cprestamos_personales_slope_3,mcuenta_corriente_slope_3
0,249328966,202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249328966,202102,1.0,-1.0,31382.320,0.00,0.0,-722.810,0.0,-42174.860,-4696.750,0.0,0.0,0.0,13342.480,0.0,0.0
2,249328966,202103,4.5,-0.5,12318.755,0.00,0.0,-426.685,0.0,-22748.310,-2106.365,0.0,0.0,0.0,-2545.440,0.0,0.0
3,249328966,202104,3.2,-0.3,7550.850,0.00,0.0,-118.476,0.0,-9417.598,1.047,0.0,0.0,0.0,5269.316,0.0,0.0
4,249328966,202105,0.8,0.0,1252.336,0.00,0.0,-47.606,0.0,-1781.722,-3045.928,0.0,0.0,0.0,805.637,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1595193701,202105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981942,1595193701,202106,36.0,0.0,599.300,-75899.52,-1.0,558.760,0.0,1961.900,24405.880,0.0,-1.0,0.0,0.000,0.0,0.0
981943,1598206246,202106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981944,1598444941,202106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
nuevos_features = ""
for campo in campos:
  nuevos_features +=  f"\n    ,percent_rank() OVER (ORDER BY {campo}) as percent_rank_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
from competencia_01

"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    ,percent_rank() OVER (ORDER BY ctrx_quarter) as percent_rank_ctrx_quarter
    ,percent_rank() OVER (ORDER BY cpayroll_trx) as percent_rank_cpayroll_trx
    ,percent_rank() OVER (ORDER BY mpayroll) as percent_rank_mpayroll
    ,percent_rank() OVER (ORDER BY mpayroll2) as percent_rank_mpayroll2
    ,percent_rank() OVER (ORDER BY cpayroll2_trx) as percent_rank_cpayroll2_trx
    ,percent_rank() OVER (ORDER BY mpasivos_margen) as percent_rank_mpasivos_margen
    ,percent_rank() OVER (ORDER BY mprestamos_personales) as percent_rank_mprestamos_personales
    ,percent_rank() OVER (ORDER BY mcuentas_saldo) as percent_rank_mcuentas_saldo
    ,percent_rank() OVER (ORDER BY mcaja_ahorro) as percent_rank_mcaja_ahorro
    ,percent_rank() OVER (ORDER BY mcomisiones_mantenimiento) as percent_rank_mcomisiones_mantenimiento
    ,percent_rank() OVER (ORDER BY cproductos) as percent_rank_cproductos
   

In [21]:
%%sql
CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    ,percent_rank() OVER (ORDER BY ctrx_quarter) as percent_rank_ctrx_quarter
    ,percent_rank() OVER (ORDER BY cpayroll_trx) as percent_rank_cpayroll_trx
    ,percent_rank() OVER (ORDER BY mpayroll) as percent_rank_mpayroll
    ,percent_rank() OVER (ORDER BY mpayroll2) as percent_rank_mpayroll2
    ,percent_rank() OVER (ORDER BY cpayroll2_trx) as percent_rank_cpayroll2_trx
    ,percent_rank() OVER (ORDER BY mpasivos_margen) as percent_rank_mpasivos_margen
    ,percent_rank() OVER (ORDER BY mprestamos_personales) as percent_rank_mprestamos_personales
    ,percent_rank() OVER (ORDER BY mcuentas_saldo) as percent_rank_mcuentas_saldo
    ,percent_rank() OVER (ORDER BY mcaja_ahorro) as percent_rank_mcaja_ahorro
    ,percent_rank() OVER (ORDER BY mcomisiones_mantenimiento) as percent_rank_mcomisiones_mantenimiento
    ,percent_rank() OVER (ORDER BY cproductos) as percent_rank_cproductos
    ,percent_rank() OVER (ORDER BY cdescubierto_preacordado) as percent_rank_cdescubierto_preacordado
    ,percent_rank() OVER (ORDER BY mtarjeta_visa_consumo) as percent_rank_mtarjeta_visa_consumo
    ,percent_rank() OVER (ORDER BY cprestamos_personales) as percent_rank_cprestamos_personales
    ,percent_rank() OVER (ORDER BY mcuenta_corriente) as percent_rank_mcuenta_corriente
from competencia_01

,Success


In [ ]:
## cree 5 tablas
## las sumas que me parecian mas unos ratios..
## otro con lag y delta de dos meses
## promedio de todos los campos
## regr_slope de todos los campos
## per_Rank

##.. ahora tengo que unir esto

In [22]:
%%sql
SELECT * FROM competencia_01_percent_rank 


,numero_de_cliente,foto_mes,percent_rank_ctrx_quarter,percent_rank_cpayroll_trx,percent_rank_mpayroll,percent_rank_mpayroll2,percent_rank_cpayroll2_trx,percent_rank_mpasivos_margen,percent_rank_mprestamos_personales,percent_rank_mcuentas_saldo,percent_rank_mcaja_ahorro,percent_rank_mcomisiones_mantenimiento,percent_rank_cproductos,percent_rank_cdescubierto_preacordado,percent_rank_mtarjeta_visa_consumo,percent_rank_cprestamos_personales,percent_rank_mcuenta_corriente
0,289831568,202101,0.000000,0.0,0.0,0.0,0.0,0.146885,0.0,0.000006,0.000069,0.014346,0.000050,0.000000,0.000000,0.0,0.000000
1,289831568,202102,0.000000,0.0,0.0,0.0,0.0,0.065683,0.0,0.000006,0.000069,0.014346,0.000050,0.000000,0.000000,0.0,0.000000
2,689771532,202106,0.976330,0.0,0.0,0.0,0.0,0.963382,0.0,0.993421,0.080752,0.998454,0.986650,0.035647,0.764758,0.0,0.000002
3,689771532,202104,0.981029,0.0,0.0,0.0,0.0,0.794183,0.0,0.979626,0.000069,0.721082,0.986650,0.035647,0.947732,0.0,0.000003
4,689771532,202105,0.978356,0.0,0.0,0.0,0.0,0.951430,0.0,0.000001,0.000069,0.721082,0.961722,0.035647,0.780747,0.0,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,989634931,202103,0.986409,0.0,0.0,0.0,0.0,0.999965,0.0,0.999486,0.985968,0.765455,0.775330,0.035647,0.538082,0.0,0.999996
981942,655948616,202101,0.999918,0.0,0.0,0.0,0.0,0.999970,0.0,0.999624,0.739810,0.728068,0.986650,0.035647,0.969617,0.0,0.999997
981943,342400020,202101,0.999846,0.0,0.0,0.0,0.0,0.999973,0.0,0.999645,0.989226,0.998593,0.546294,0.035647,0.987380,0.0,0.999998
981944,255255678,202104,0.999979,0.0,0.0,0.0,0.0,0.999995,0.0,0.999748,0.902411,0.721082,0.986650,0.035647,0.999908,0.0,0.999999


In [6]:
%%sql 
create or replace table competencia_01_fe as
SELECT * FROM competencia_01 as base

LEFT JOIN competencia_01_suma_division as suma
ON base.numero_de_cliente = suma.numero_de_cliente 
    AND base.foto_mes = suma.foto_mes  

LEFT JOIN competencia_01_lag_delta as lag
ON suma.numero_de_cliente = lag.numero_de_cliente 
    AND suma.foto_mes = lag.foto_mes  

LEFT JOIN competencia_01_avg as avg 
ON suma.numero_de_cliente = avg.numero_de_cliente 
    AND suma.foto_mes = avg.foto_mes  
    
LEFT JOIN competencia_01_regr_slope as reg 
ON suma.numero_de_cliente = reg.numero_de_cliente 
    AND suma.foto_mes = reg.foto_mes

LEFT JOIN competencia_01_percent_rank as p_rank
ON suma.numero_de_cliente = p_rank.numero_de_cliente 
    AND suma.foto_mes = p_rank.foto_mes

,Success


In [4]:
%%sql
CREATE OR REPLACE TABLE competencia_01_fe AS
SELECT numero_de_cliente, foto_mes
FROM competencia_01 
LEFT JOIN competencia_01_suma_division 
USING (numero_de_cliente, foto_mes)
LEFT JOIN competencia_01_lag_delta 
USING (numero_de_cliente, foto_mes)
LEFT JOIN competencia_01_avg 
USING (numero_de_cliente, foto_mes)
LEFT JOIN competencia_01_regr_slope 
USING (numero_de_cliente, foto_mes)
LEFT JOIN competencia_01_percent_rank 
USING (numero_de_cliente, foto_mes)

,Success


In [23]:
# %%sql
# CREATE OR REPLACE TABLE competencia_01_fe AS
# SELECT *
# FROM competencia_01_suma_division as suma
# LEFT JOIN competencia_01_lag_delta as lag
# USING (numero_de_cliente, foto_mes)
# LEFT JOIN competencia_01_avg as avg
# USING (numero_de_cliente, foto_mes)
# LEFT JOIN competencia_01_regr_slope as reg
# USING (numero_de_cliente, foto_mes)
# LEFT JOIN competencia_01_percent_rank as p_rank
# USING (numero_de_cliente, foto_mes)

,Success


In [7]:
%%sql
COPY competencia_01_fe TO '../../../datasets/competencia_01_fe.csv' (FORMAT CSV, HEADER TRUE)

,Success


In [8]:
%%sql
select 
    foto_mes,foto_mes_1
from competencia_01_fe
limit 2

,foto_mes,foto_mes_1
0,202106,202106
1,202106,202106
